In [1]:
# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

# Model Selection, Metrics, and Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Data wrangling
import pandas as pd
import numpy as np
import re

# Time
import time
import datetime
from datetime import datetime as dt
import dateutil.parser

# Plotting
import seaborn as sns
from seaborn import plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as pplt

# URL Grabbing
import requests

# Scraping / Searching
from bs4 import BeautifulSoup
import re

# Misc
import pickle
from pprint import pprint

# %matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context('poster')
pd.set_option('display.float_format', lambda x: '%.2f' % x) # reduces the amount of decimals when printing the df's

In [2]:
# with open("movies.pkl", 'rb') as picklefile: 
#     df = pickle.load(picklefile)
    
with open("crew.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)

In [3]:
# take a peek into the data
df.head().T

,0,1,2,3,4
index,The Other Side of the Mountain Part II,Universal Soldier,Newtown,Mommie Dearest,G.B.F.
actors,NaN,"[Jean-Claude Van Damme, Dolph Lundgren]",NaN,NaN,"[Natasha Lyonne, Megan Mullally]"
close_date,NaT,NaT,NaT,NaT,NaT
composers,NaN,[Tim Simonec],NaN,NaN,NaN
days_in_theater,NaN,NaN,14.00,NaN,NaN
directors,NaN,[Roland Emmerich],NaN,NaN,NaN
domestic_gross_adj,42753500.00,75661200.00,13900.00,59219100.00,NaN
foreign_unadj,NaN,NaN,NaN,NaN,NaN
genre,Romance,Sci-Fi Action,Documentary,Drama,Comedy
mpaa_rating,PG,R,Unrated,PG,R


# Feature Selection

In [4]:
# adding in the months of release
df['release_month'] = df.release_date.dt.month

In [5]:
# dropping all the multidimensional data, that will be engineered later`
df_domestic_pred = df.drop(['actors', 'composers', 'directors', 'producers', 'writers'], 1)

# drop all the unadjusted data
df_domestic_pred = df_domestic_pred.drop(['foreign_unadj', 'worldwide_gross_unadj'], 1)

# dropping all edge cases
df_domestic_pred = df_domestic_pred[(df_domestic_pred.domestic_gross_adj > 10000) & # drop all with significantly low grosses
                                    (df_domestic_pred.mpaa_rating != 'Unrated') & # kills all unrated
                                    (df_domestic_pred.runtime >= 40) & # drop all movies less than feature length
                                    (df_domestic_pred.release_date.dt.year >= 1990)] # remove all unrated

# dropping highly correlated data (and intertwined)
df_domestic_pred = df_domestic_pred.drop(['opening_weekend_adj'], 1)

In [6]:
print('Number of rows:',len(df_domestic_pred))
df_domestic_pred.head().T

Number of rows: 5496


,1,5,6,7,8
index,Universal Soldier,Saw IV,Better Than Chocolate,The Jackal,Murderball
close_date,NaT,2007-12-13 00:00:00,NaT,NaT,2005-10-06 00:00:00
days_in_theater,NaN,49.00,NaN,NaN,91.00
domestic_gross_adj,75661200.00,79585200.00,3431700.00,103517800.00,2066200.00
genre,Sci-Fi Action,Horror,Romantic Comedy,Action Thriller,Documentary
mpaa_rating,R,R,R,R,R
number_of_theaters,1925.00,3183.00,130.00,2309.00,97.00
production_budget_adj,NaN,NaN,NaN,60000000.00,NaN
release_date,1992-07-10 00:00:00,2007-10-26 00:00:00,1999-08-13 00:00:00,1997-11-14 00:00:00,2005-07-08 00:00:00
runtime,102.00,108.00,102.00,125.00,88.00


In [7]:
# %matplotlib inline
sns.pairplot(df_domestic_pred.dropna(), size=4);

In [8]:
# checking the correlations for any intracorrelations
sns.set_context('poster')
sns.heatmap(df_domestic_pred.corr());

# Transformation  
  
Here we take a stab at transforming all the data into a normal form

In [9]:
sns.set_context('notebook')

### Before Transformation

In [10]:
# Checking if any values need to be transformed
df_domestic_pred.hist(figsize=(10,6));

Production Budget (log)

In [11]:
df_domestic_pred['production_budget_sqrt'] = df_domestic_pred['production_budget_adj'].apply(np.sqrt)
df_domestic_pred.drop(['production_budget_adj'], 1);
# df_domestic_pred['production_budget_log'].hist();

Days In Theater (log)

In [12]:
df_domestic_pred['days_in_theater_log'] = df_domestic_pred['days_in_theater'].apply(np.log)
df_domestic_pred.drop(['days_in_theater'], 1);
# df_domestic_pred['days_in_theater_log'].hist();

Runtime (log)

In [13]:
# df_domestic_pred['runtime'].apply(np.log).hist()
df_domestic_pred['runtime_log'] = df_domestic_pred['runtime'].apply(np.log)
df_domestic_pred.drop(['runtime'], 1);
# df_domestic_pred['runtime_log'].hist();

In [14]:
# df_domestic_pred['made_men_log'] = df_domestic_pred['made_men'].apply(np.log)
# df_domestic_pred.drop(['days_in_theater'], 1)
# df_domestic_pred['days_in_theater_log'].hist();
# df_domestic_pred[df_domestic_pred.made_men == np.nan]
# df_domestic_pred['made_men'].hist();

#### Cleaning

In [15]:
# choose the predicted columns
df_domestic_pred = df_domestic_pred[['domestic_gross_adj', 'production_budget_sqrt', 'made_men',
                                     'runtime_log', 'days_in_theater_log', 'number_of_theaters',
                                     'release_month']]

In [16]:
# kill off any NA 
df_domestic_pred = df_domestic_pred.dropna()#subset=['domestic_gross_adj', 'production_budget_log', 
                                            #       'runtime_log', 'days_in_theater_log', 'number_of_theaters'])

In [17]:
print('Number of Films after Filtering:',len(df_domestic_pred))
df_domestic_pred.head()

Number of Films after Filtering: 1437


,domestic_gross_adj,production_budget_sqrt,made_men,runtime_log,days_in_theater_log,number_of_theaters,release_month
10,195732200.00,2236.07,14,4.70,4.34,3807.00,8.00
11,55815700.00,4472.14,2,4.70,4.34,2762.00,6.00
18,18664000.00,7745.97,9,4.62,4.25,2215.00,11.00
25,176014600.00,6557.44,7,4.76,5.12,3184.00,6.00
32,352178200.00,8602.33,11,4.51,5.08,4311.00,7.00


In [19]:
len(df_domestic_pred)

1437

### After Transformation

In [20]:
# df_domestic_pred.hist(figsize=(10,6));

# Splitting & Preparing the Data

Avoiding normalization for now, will be toggled in our models

In [21]:
# Holding onto the df for any use
X_df = df_domestic_pred.iloc[:, 1:]
y_df = df_domestic_pred.iloc[:, 0]

# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = np.array(X_df)

# Choose the response variable(s)
y = np.array(y_df)

In [22]:
# train / test splits
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [23]:
print('shapes:')
print('X_train:',X_train.shape)
print('X_test :',X_test.shape)
print('y_train:',y_train.shape)
print('y_test :',y_test.shape)

shapes:
X_train: (1077, 6)
X_test : (360, 6)
y_train: (1077,)
y_test : (360,)


In [24]:
# # first normalize our features
# X = preprocessing.normalize(X_train, axis=0)

# # turn back in DF
# df_domestic_pred_norm = pd.DataFrame(domestic_pred_norm, columns=df_domestic_pred.columns)

# Plotting Functions

In [25]:
def plot_approximation(est, ax, label=None):
    """Plot the approximation of ``est`` on axis ``ax``. """
    ax.scatter(X_train[:,0], y_train, s=50)
    ax.plot(x_plot, est.predict(x_plot), color='red', label=label)
    ax.set_ylim(-10000000, max(y))
#     ax.set_xlim(0, max(X[0]))
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.legend(loc='upper right',frameon=True)

In [26]:
def plot_coefficients(est, ax, label=None, yscale='log'):
    """A helper function to plot the absolute value of the coefficients on the right-hand column plot
    """
    coef = est.steps[-1][1].coef_.ravel() # i have 0 idea how this works
    if yscale == 'log':
        ax.semilogy(np.abs(coef), marker='o', label=label)
        ax.set_ylim((1e-1, 1e8))
    else:
        ax.plot(np.abs(coef), marker='o', label=label)
    ax.set_ylabel('abs(coefficient)')
    ax.set_xlabel('coefficients')
    ax.set_xlim((0, 10))
    ax.set_ylim(-1000,1e6)

In [27]:
def plot_residuals(est, features=X_test, col=0, color=sns.color_palette()[0], label="Estimator",_plt=None):
    if not _plt:
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes()
        ax.plot(features[:,col], y_test - est.predict(features), 
                marker='o', color=color, ls='', markersize=7, label=label)
        ax.set_xlabel('Production Budget (sqrt)')
        ax.set_ylabel('Residuals');
        return ax
    else:
        _plt.plot(features[:,col], y_test - est.predict(features),
                  marker='o', color=color, ls='', markersize=7, label=label, alpha=0.7)
        return _plt

In [28]:
def plot_actual_predicted(est, features=X_test, color=sns.color_palette()[0], label="Estimator", _plt=None):
    if not _plt:
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes()
        line = np.linspace(y_test.min(), y_test.max(), 360)

        ax.plot(y_test, est.predict(features), marker='o', ls='', markersize=7, label=label)
        ax.plot(line, line, color=sns.color_palette()[4],zorder=10)

        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted');

        min_val = min(ax.get_xlim()[0],ax.get_ylim()[0])
        max_val = max(ax.get_xlim()[1],ax.get_ylim()[1])
        ax.set_xlim(min_val, max_val)
        ax.set_ylim(min_val, max_val)

        return ax
    else:
        _plt.plot(y_test, est.predict(features), marker='o', ls='', markersize=7, label=label, alpha=0.7)
        return _plt

In [29]:
def plot_poly_error(est, order):
    train_error = np.empty(order)
    test_error = np.empty(order)
    for degree in range(order):
        model = make_pipeline(PolynomialFeatures(degree), est)
        model.fit(X_train, y_train)
        train_error[degree] = mean_squared_error(y_train, model.predict(X_train))
        test_error[degree] = mean_squared_error(y_test, model.predict(X_test))

    # Plot the training and test errors against degree
    plt.figure(figsize=(8,6))
    plt.plot(np.arange(order), train_error, color='green', label='train')
    plt.plot(np.arange(order), test_error, color='red', label='test')
    plt.ylabel('log(mean squared error)')
    plt.xlabel('degree')
    plt.xticks(range(order))
    plt.legend(loc='upper left')

In [30]:
def linear_Matrix(matrix):
    """Returns a linearly spaced numpy matrix based on the same size of the input matrix.
    Matrix must be larger than 1 column.
    """
    m, n = matrix.shape
    n_matrix = np.zeros((m,n))
    for i in range(n):
        n_matrix[:,i] = np.linspace(matrix[:,i].min(), matrix[:,i].max() + 1, m)
    return n_matrix

In [31]:
# Building an m x n dimensional matrix
x_plot = linear_Matrix(X_train)

# Building the Predictive Model

### Parameters

## Linear Regression (No Regularization)

Ordinary least squares Linear Regression.  
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

In [32]:
# Create an empty model
linear = LinearRegression(normalize=True)

# Fit the model to the full dataset
linear.fit(X_train, y_train)

# store the prediction
y_pred = linear.predict(X_test)

# Print out the metrics for the model against the full dataset
print('R^2       :',linear.score(X_train,y_train))
print('MSE (log) :',np.log(mean_squared_error(y_pred, y_test)))

R^2       : 0.509793563727
MSE (log) : 36.0026097379


In [33]:
%matplotlib notebook

In [ ]:
sns.set_context('poster')
ax = plot_residuals(linear);
ax.set_title('Linear Regression Residuals');

In [ ]:
ax = plot_actual_predicted(linear)
ax.set_title('Actual vs. Predicted (Linear Regression)')
ax.set_xlim(-1e7,ax.get_xlim()[1]);

Makes a 3D PLOT BRO

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.tick_params(direction='out')

line_matrix = linear_Matrix(np.array([X[:,0], y, X[:,4]]).T)

ax.plot(x_plot[:,0], linear.predict(x_plot), x_plot[:,4] )
ax.scatter(X_train[:,0], y_train, X_train[:,4], c='r', marker='o')

ax.set_xlabel('Production Budget (sqrt)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

plt.show()

In [ ]:
# %matplotlib inline

In [ ]:
sns.set_context('poster')

# Checking the model's shape against the heaviest correlated value
plt.scatter( X[:,0], y, marker='.')
plt.plot( x_plot.T[0].T, linear.predict(x_plot), linewidth=2, color='k' )
ax = plt.gca()

ax.set_title('Actual vs. Predicted (Linear Regression)')
ax.set_xlabel('Production Budget (sqrt)')
ax.set_ylabel('Domestic Gross');
ax.set_ylim(-10000000, max(y));
# ax.set_xlim(-0.01, max(x_plot[0]))

## Polynomial Linear Regression

Some features look good here..

In [34]:
# Make the model
poly_linear = make_pipeline(PolynomialFeatures(3), LinearRegression(normalize=True))
poly_linear.fit(X_train, y_train)

# Save some test features and do a prediction
X_test_poly = preprocessing.normalize(X_test, axis=0)
poly_y_pred = poly_linear.predict(X_test_poly)

# Print out the metrics for the model against the full dataset
print('R^2       :',poly_linear.score(X_train,y_train))
print('MSE (log) :',np.log(mean_squared_error(poly_y_pred, y_test)))

R^2       : 0.756220608363
MSE (log) : 47.193139645


In [35]:
pd.DataFrame(list(zip(X_df.columns,poly_linear.steps[1][-1].coef_)), columns=['features', 'coefficients'])

,features,coefficients
0,production_budget_sqrt,-0.04
1,made_men,-998360.69
2,runtime_log,-400680276.10
3,days_in_theater_log,15493097191.45
4,number_of_theaters,-5231939171.77
5,release_month,7657732.12


In [ ]:
ax = plot_residuals(linear, label="Linear");
ax.set_title('3rd Degree Polynomial Linear Regression Residuals');
plot_residuals(poly_linear, _plt=ax, color=sns.color_palette()[1], label="Poly Linear").legend();

In [ ]:
ax = plot_actual_predicted(linear, label="Linear")
ax.set_xlim(-1e7,ax.get_xlim()[1]);
ax.set_title('Actual vs. Predicted');
plot_actual_predicted(poly_linear, _plt=ax, color=sns.color_palette()[0], label="Poly Linear").legend();

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.tick_params(direction='out')

line_matrix = linear_Matrix(np.array([X[:,0], y, X[:,4]]).T)

ax.plot(x_plot[:,0], poly_linear.predict(x_plot), x_plot[:,4], c=sns.color_palette()[1], label='Linear')
ax.plot(x_plot[:,0], linear.predict(x_plot),      x_plot[:,4], c=sns.color_palette()[2], label='Poly Linear')
ax.scatter(X_train[:,0], y_train, X_train[:,4],                c=sns.color_palette()[0], marker='o')

ax.set_xlabel('Production Budget (sqrt)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

ax.set_title('Linear vs. 3rd Order Polynomial Linear Regression');

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

plt.show();

In [ ]:
# Checking the model's shape against the heaviest correlated value
sns.set_context('poster')

fig = plt.figure(figsize=(10,10))
ax = plt.axes()


plt.scatter(X[:,0], y, marker='.')
# plt.plot( x_plot.T[0].T, poly_linear.predict(x_plot), linewidth=2, color='k' )
plt.plot(x_plot[:,0], poly_linear.predict(x_plot), c=sns.color_palette()[1], label='Poly Linear')
plt.plot(x_plot[:,0], linear.predict(x_plot),      c=sns.color_palette()[2], label='Linear')
ax = plt.gca()
ax.set_ylim(-10000000, max(y))

ax.set_title('Linear vs. 3rd Order Polynomial Linear Regression');
ax.set_xlabel('Production Budget (sqrt)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.legend();


In [ ]:
plot_poly_error(LinearRegression(normalize=True), order=10)

In [ ]:
# Step through degrees from 0 to 9 and store the training and test (generalization) error.
# create array of zeros,
train_error = np.empty(10)
test_error = np.empty(10)
for degree in range(10):
    poly_linear = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_linear.fit(X_train, y_train)
    train_error[degree] = mean_squared_error(y_train, poly_linear.predict(X_train))
    test_error[degree] = mean_squared_error(y_test, poly_linear.predict(X_test))

# Plot the training and test errors against degree
plt.figure(figsize=(8,6))
plt.plot(np.arange(10), train_error, color=sns.color_palette()[0], label='train')
plt.plot(np.arange(10), test_error,  color=sns.color_palette()[1], label='test')
plt.ylabel('log(mean squared error)', labelpad=-3)
plt.xlabel('Polynomial Degree')
plt.title('Error of Linear Polynomial Models')
plt.legend(loc='upper left');

To address the high bias of the degrees of 2 or 3 degrees looks like a good start

I'm beginning to understand the struggle to get the right model!  
Let's try passing this to a lasso

## Lasso

In [ ]:
np_lasso = LassoCV(normalize=True, cv=5)
np_lasso = np_lasso.fit(X_train, y_train)

y_pred = np_lasso.predict(X_test)

In [ ]:
plot_actual_predicted(np_lasso)

### Checking lasso effects 

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.tick_params(direction='out')

line_matrix = linear_Matrix(np.array([X[:,0], y, X[:,4]]).T)

ax.plot(x_plot[:,0], np_lasso.predict(x_plot), x_plot[:,4] )
ax.scatter(X_train[:,0], y_train, X_train[:,4], c='r', marker='o')

ax.set_xlabel('Production Budget (log)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

ax.view_init(elev=20., azim=-180)

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.tick_params(direction='out')

ax.plot(x_plot[:,0], np_lasso.predict(x_plot), x_plot[:,4] )
ax.scatter(X_train[:,0], y_train, X_train[:,4], c='r', marker='o')



ax.set_xlabel('Production Budget (log)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())



plt.show()

## Polynomial Lasso  
Least absolute shrinkage and selection operator  
https://en.wikipedia.org/wiki/Lasso_(statistics)

In [ ]:
# Step through degrees from 0 to 9 and store the training and test (generalization) error.
# create array of zeros,
train_error = np.empty(6)
test_error = np.empty(6)
for degree in range(6):
    poly_lasso = make_pipeline(PolynomialFeatures(degree), LassoCV(normalize=True, cv=3))
    poly_lasso.fit(X_train, y_train)
    train_error[degree] = mean_squared_error(y_train, poly_lasso.predict(X_train))
    test_error[degree] = mean_squared_error(y_test, poly_lasso.predict(X_test))

# Plot the training and test errors against degree
plt.figure(figsize=(8,6))
plt.plot(np.arange(6), train_error, color=sns.color_palette()[0], label='train')
plt.plot(np.arange(6), test_error,  color=sns.color_palette()[1], label='test')
plt.ylabel('log(mean squared error)', labelpad=-3)
plt.xlabel('Polynomial Degree')
plt.title('Error of Lasso Polynomial Models')
plt.legend(loc='upper left');

In [ ]:
degrees = 3
pf = PolynomialFeatures(degrees)

X_train_pf = pf.fit_transform(X_train)
X_test_pf =  pf.transform(X_test)

lasso_plot = linear_Matrix(X_train_pf)

lasso = LassoCV(normalize=True, cv=5)
lasso = lasso.fit(X_train_pf, y_train)

y_pred = lasso.predict(X_test_pf)
lasso.score(X_train_pf, y_train)

# Print out the metrics for the model against the full dataset
print('R^2       :',lasso.score(X_train_pf,y_train))
print('MSE (log) :',np.log(mean_squared_error(y_pred, y_test)))

In [ ]:
ax = plot_residuals(linear, label="Linear");
plot_residuals(poly_linear, _plt=ax,               color=sns.color_palette()[1], label="Poly Linear")

# adjust column it's pulling in the X axis for since X_test_pf includes the intercept
# you can see the features here: pf.get_feature_names()
plot_residuals(lasso, features=X_test_pf, col=1,_plt=ax, color=sns.color_palette()[2], label="Poly Lasso")

ttl = ax.set_title('Linear Regression, 3rd Degree Polynomial Linear & Lasso Regression Residuals');
ttl.set_position([.5, 1.05])
ax.legend();

In [ ]:
ax = plot_actual_predicted(linear, label="Linear")
plot_actual_predicted(poly_linear, _plt=ax, color=sns.color_palette()[0], label="Poly Linear")
plot_actual_predicted(lasso      , _plt=ax, color=sns.color_palette()[0], label="Poly Linear", features=X_test_pf)

ax.legend();
ax.set_xlim(-1e7,ax.get_xlim()[1]);
ax.set_title('Actual vs. Predicted');

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.tick_params(direction='out')

line_matrix = linear_Matrix(np.array([X[:,0], y, X[:,4]]).T)

ax.plot(x_plot[:,0], poly_linear.predict(x_plot),  x_plot[:,4],     c=sns.color_palette()[1], label='Linear')
ax.plot(x_plot[:,0], linear.predict(x_plot),       x_plot[:,4],     c=sns.color_palette()[2], label='Poly Linear')
ax.plot(lasso_plot[:,1], lasso.predict(lasso_plot),lasso_plot[:,5], c=sns.color_palette()[3], label='Lasso')
ax.scatter(X_train[:,0], y_train, X_train[:,4],                c=sns.color_palette()[0], marker='o')

ax.set_xlabel('Production Budget (sqrt)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

ax.set_title('Linear vs. 3rd Order Polynomials');
ax.legend()

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

plt.show();

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.tick_params(direction='out')

line_matrix = linear_Matrix(np.array([X[:,0], y, X[:,4]]).T)

# ax.plot(x_plot[:,0], poly_linear.predict(x_plot),  x_plot[:,4],     c=sns.color_palette()[1], label='Linear')
ax.plot(x_plot[:,0], linear.predict(x_plot),       x_plot[:,4],     c=sns.color_palette()[2], label='Poly Linear')
ax.plot(lasso_plot[:,1], lasso.predict(lasso_plot),lasso_plot[:,5], c=sns.color_palette()[3], label='Lasso')
ax.scatter(X_train[:,0], y_train, X_train[:,4],                     c=sns.color_palette()[0], marker='o')

ax.set_xlabel('Production Budget (sqrt)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

ax.set_title('Linear vs. 3rd Order Polynomials');
ax.legend()

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

plt.show();

In [ ]:
sns.set_context('poster')

# Checking the model's shape against the heaviest correlated value
plt.scatter( X[:,0], y, marker='.')
plt.plot( lasso_plot[:,1], lasso.predict(lasso_plot), linewidth=2, color='k' )
ax = plt.gca()

ax.set_title('Budget vs. Gross (Polynomial Lasso Regression)')
ax.set_xlabel('Production Budget (sqrt)')
ax.set_ylabel('Domestic Gross');
ax.set_ylim(-10000000, max(y));
# ax.set_xlim(-0.01, max(x_plot[0]))

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.axes()
line = np.linspace(y_test.min(), y_test.max(), 360)

ax.plot(y_test, y_pred, marker='o', ls='', ms=5)
ax.plot(line, line)

min_val = min(ax.get_xlim()[0],ax.get_ylim()[0])
max_val = max(ax.get_xlim()[1],ax.get_ylim()[1])
ax.set_xlim(min_val, max_val)
ax.set_ylim(min_val, max_val);

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(lasso_plot[:,1], lasso.predict(lasso_plot), lasso_plot[:,5] )
ax.scatter(X_train_pf[:,1], y_train, X_train_pf[:,5], c='r', marker='o')

ax.tick_params(direction='out')

ax.set_xlabel('Production Budget (log)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

ax.view_init(elev=20., azim=-180)

plt.show()

In [ ]:
# # Checking the model's shape against the heaviest correlated value
# plot_matrix = preprocessing.normalize(linear_Matrix(X_test_pf))
# plot_pred = preprocessing.normalize(lasso.predict(plot_matrix).T)

# plt.scatter(preprocessing.normalize(X_test[:,0]), preprocessing.normalize(y_test), marker='.')
# plt.plot(plot_matrix[:,0] , plot_pred, linewidth=2, color='k' )
# ax = plt.gca()

In [ ]:
sorted(lasso.coef_, key=abs, reverse=True)

In [ ]:
lasso.alpha_, lasso.coef_

In [ ]:
np.log(mean_squared_error(y_test, y_pred))

#### Revisit ...

In [ ]:
plot_poly_error(LassoCV(normalize=True, cv=5), 5)

In [ ]:
# Looking at above, the lower the penalty on the coefficients, the better?
poly_lasso.set_params(lasso__alpha=1e-8)
np.mean(y_test - poly_lasso.predict(X_test))

In [ ]:
# # Checking the model's shape against the heaviest correlated value
# plt.scatter(X_train[:,0], y_train, marker='.') # production budget vs domestic gross
# plt.plot( x_plot.T[0].T, lasso.predict(X_train), linewidth=2, color='k' ) # plot against data
# ax = plt.gca() 
# ax.set_ylim(-0.01, 0.19)
# ax.set_xlim(-0.01, 0.065)

In [ ]:
lasso

In [ ]:
# this determines the best coefficients 
all_scores = []

for train, test in kf.split(X):
    x_tr = X.iloc[train]
    y_tr = y.iloc[train]
    x_te = X.iloc[test]
    y_te = y.iloc[test]
    mse_score = []
    for i in alphas:
        lasso.set_params(lasso__alpha=i)
        lasso.fit(x_tr,y_tr)
        mse = np.mean((y_te - lasso.predict(x_te))**2)
        mse_score.append(mse)
    all_scores.append(mse_score)

zipped = zip(X.columns, lasso.coef_)
features = sorted(zipped, key = lambda x: abs(x[1]), reverse=True)
for f in features[:]:
    print ("{:20.20s} {:7.3f}".format(f[0], f[1]))

## Ridge Regression


http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV

In [ ]:
plot_poly_error(RidgeCV(normalize=True, cv=5), 5);

In [ ]:
# Set up a figure and axes for 8 plots, 2 per row for 4 rows
fig, ax_rows = plt.subplots(4, 2, figsize=(15, 20))

degree = 2
alphas = [1e-5, 1, 10, 100]
for alpha, ax_row in zip(alphas, ax_rows):
    ax_left, ax_right = ax_row
    est = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=alpha, normalize=True))
    est.fit(X_train, y_train)
    plot_approximation(est, ax_left, label='alpha=%r' % alpha)
    plot_coefficients(est, ax_right, label='Ridge(alpha=%r) coefficients' % alpha)

plt.tight_layout()

## Elastic Net

In [ ]:
%matplotlib inline

In [ ]:
enet = ElasticNetCV(alphas = [1e-5,1e-3,1e-1,1,100],
                    l1_ratio = [1e-5,1e-3,1e-1,0.1,0.9,0.5],
                    normalize=True, 
                    cv=5,
                    random_state=9)
enet = enet.fit(X_train, y_train)

y_pred = enet.predict(X_test)

In [ ]:
poly_enet = make_pipeline(PolynomialFeatures(2), 
                          ElasticNetCV(alphas = [1e-5,1e-3,1e-1,1,100],
                                       l1_ratio = [1e-8,1e-5,1e-3,1e-1,0.1,0.9,0.5],
                                       normalize=True, 
                                       cv=5,
                                       random_state=9))
poly_enet.fit(X_train, y_train)
X_train_pf =  poly_enet.steps[0][-1].transform(X_train)
poly_enet_plot = linear_Matrix(X_train)

poly_enet.score(X_train, y_train)

In [ ]:
print('Best Alpha:', enet.alpha_)
print('Ratio:', enet.l1_ratio_)
print('MSE (log):', np.log(mean_squared_error(y_test,enet.predict(X_test))))
print('Top 20 Coefficients:')
pprint(sorted(poly_enet.steps[1][-1].coef_, key=abs, reverse=True)[:20])

In [ ]:
plot_residuals(poly_enet);

In [ ]:
plot_actual_predicted(poly_enet)

In [ ]:
sns.set_context('poster')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# ax.plot(x_plot[:,1], poly_enet.predict(x_plot), x_plot[:,5] )
ax.scatter(X_train[:,1], y_train, X_train[:,5], c='r', marker='o')

ax.tick_params(direction='out')

ax.set_xlabel('Production Budget (log)', labelpad=30)
ax.set_ylabel('Domestic Gross', labelpad=30)
ax.set_zlabel('Number of Theaters', labelpad=30)

# ax.set_xlim(X_train[:,0].min(), X_train[:,0].max())
# ax.set_ylim(y.min(), y.max())
ax.set_zlim(X_train[:,4].min(), X_train[:,4].max())

# ax.view_init(elev=20., azim=-180)

plt.show()